<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Working%20RNN%20/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dense


In [13]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Loading dataframe on current active/and closed coal mines

In [170]:
#join in overall info
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')
mines = pd.DataFrame(mines)
mines = mines[['company',
               'coalmine_op_status',
               'mine_name',
               'permit_id',
               'state_tribe_mine_id',
               'national_id',
               'coal_bed_names',
               'inspectable_unit_status',
                'post_smcra',
               'reported_area',
               'permit_application_type',
               'permit_application_date',
               'permit_approval_date',
               'edit_date',
               'area_mine',
               'contour',
               'mountaintop',
               'steep_slope',
               'highwall',
               'auger',
               'contact']].drop_duplicates()

#replace empty strings with NAN
mines = mines.replace(r'^\s*$', np.nan, regex=True)

### Loading dataframes from google earth engine queries

In [15]:
import pandas as pd
import os

directory_path = '/content/drive/MyDrive/Mines_Data/'
directory_files = os.listdir(directory_path)

df = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['year'] = directory_files[z]
    df_file['year'] = df_file['year'].str.slice(3, 7)
    df_file['end_date'] = df_file['year']+'-07-31'
    df_file['start_date'] = df_file['year']+'-07-01'
    df_file['start_date'] = pd.to_datetime(df_file['start_date'])
    df_file['end_date'] = pd.to_datetime(df_file['end_date'])

    df_file['State'] = directory_files[z]
    df_file['State'] = df_file['State'].str.slice(0, 2)

    df = pd.concat([df, df_file])

    z+=1


### aggregating to averages of land class(the probability) classes

In [16]:

#mapping demo in python https://colab.research.google.com/github/QuantEcon/quantecon-notebooks-datascience/blob/master/applications/maps.ipynb#scrollTo=SI5i7rboSGFL&uniqifier=2
# Read the downloaded file

0	#419bdf	water, #397d49	trees, #88b053	grass, #7a87c6	flooded_vegetation, #e49635	crops, #dfc35a	shrub_and_scrub, #c4281b	built, #a59b8f	bare, #b39fe1	snow_and_ice

label0 = df[['orig_perm_id', 'Company','mine_id','water', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date','Company']).mean().reset_index()
label1 = df[['orig_perm_id', 'Company','mine_id', 'trees', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label2 = df[['orig_perm_id', 'Company','mine_id', 'grass', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label3 = df[['orig_perm_id', 'Company','mine_id','flooded_vegetation', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date', 'Company']).mean().reset_index()
label4 = df[['orig_perm_id', 'Company','mine_id', 'crops', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index().drop_duplicates()
label5 = df[['orig_perm_id', 'Company','mine_id', 'shrub_and_scrub', 'start_date']].groupby(['orig_perm_id', 'start_date', 'mine_id','Company']).mean().reset_index()
label6 = df[['orig_perm_id', 'Company','mine_id', 'built', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label7 = df[['orig_perm_id', 'Company','mine_id', 'bare', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label8 = df[['orig_perm_id', 'Company','mine_id', 'snow_and_ice', 'start_date']].groupby(['orig_perm_id','start_date','mine_id', 'Company']).mean().reset_index()

label0 =label0.drop_duplicates()
label1 =label1.drop_duplicates()
label2 =label2.drop_duplicates()
label3 =label3.drop_duplicates()
label4 =label4.drop_duplicates()
label5 =label5.drop_duplicates()
label6 =label6.drop_duplicates()
label7 =label7.drop_duplicates()
label8 =label8.drop_duplicates()

group_df = df[['orig_perm_id', 'start_date', 'mine_id', 'Company']].groupby(['orig_perm_id', 'start_date', 'mine_id']).count().reset_index()
group_df = group_df.drop_duplicates()
group_df = group_df.rename(columns={"Company": "Observations"})
df = pd.merge(df, group_df, on=['orig_perm_id', 'start_date', 'mine_id'], how='left').drop_duplicates()
df

In [19]:
df = df[['Company', 'start_date', 'Observations', 'orig_perm_id', 'mine_id']].drop_duplicates()
df = df.reset_index()
df = df.drop(['index'], axis=1)

### joining aggregated values back into main dataframe

In [20]:
df = pd.merge(df, label0, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label1, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label2, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label3, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label4, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label5, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label6, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label7, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label8, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()

df.fillna(0, inplace=True)
df

,Company,start_date,Observations,orig_perm_id,mine_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,0.031113,0.623011,0.059563,0.032779,0.065345,0.062902,0.035015,0.032943,0.036427
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,0.031869,0.698788,0.039705,0.029274,0.039489,0.036189,0.033532,0.034271,0.043253
2,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,0.032205,0.539393,0.055222,0.030944,0.088415,0.041989,0.070858,0.030897,0.037347
3,MOUNT VICTORY COAL CO INC,2023-07-01,16,404774X,00000000000000003095,0.033029,0.735718,0.038324,0.029872,0.033263,0.031299,0.029638,0.032630,0.035219
4,EVERGREEN MINING INC,2023-07-01,30,8260501,0000000000000000086a,0.030702,0.743186,0.033933,0.027343,0.029957,0.035815,0.029723,0.036228,0.032969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32655,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,0.031039,0.587889,0.039089,0.025979,0.048085,0.042070,0.107876,0.037235,0.042496
32656,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,0.030835,0.739765,0.032717,0.027254,0.030536,0.037661,0.031239,0.037154,0.032554
32657,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,0.027965,0.710045,0.042984,0.025502,0.038912,0.048706,0.032311,0.033129,0.031919
32658,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,0.036899,0.544176,0.040389,0.030385,0.046551,0.055853,0.064093,0.120093,0.039626


### Checking length before and after joining on permit_id

In [83]:
len(df)

32660

In [193]:
pd.merge(df, mines,  how='left', left_on=['orig_perm_id'], right_on = ['permit_id'])

## 23 duplicates using permit_id

### Appending the number of unique companies per permit_id to dataframe

In [171]:
Company_check = mines[['permit_id', 'company']].groupby(['permit_id']).nunique().sort_values('company', ascending=False)
Company_check = Company_check.rename(columns={"company": "Number_company_to_permit_id"})
Company_check[Company_check['Number_company_to_permit_id']>1]

,Number_company_to_permit_id
permit_id,
3049,2
2850,2
2519,2
2827,2
3013,2
2408,2
3275,2
80-156,2
3179,2


### Checking mines with different company names for the same permit_id

In [172]:
mines = pd.merge(mines, Company_check, on=['permit_id'], how='left').drop_duplicates()
mines

,company,coalmine_op_status,mine_name,permit_id,state_tribe_mine_id,national_id,coal_bed_names,inspectable_unit_status,post_smcra,reported_area,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,Number_company_to_permit_id
0,HIGGINS COAL CO INC,7,MINE #50,2404,NaN,TN2404,NaN,NaN,1.0,NaN,...,1985-06-11,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,1.0
1,MIDNIGHT COAL CO,7,MINE #1,2405,NaN,TN2405,NaN,NaN,1.0,NaN,...,1985-08-29,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,1.0
2,META ELKHORN MINING CO INC,7,AREA 11,2756,NaN,TN2756,NaN,NaN,1.0,NaN,...,1988-11-08,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,1.0
3,JORDAN BROTHERS COAL CO INC,7,TIPPLE #1,2497,NaN,TN2497,NaN,NaN,1.0,NaN,...,1991-09-09,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,1.0
4,DOUBLE Q INC,7,ANGELA #1,2284023,NaN,TN2284023,NaN,NaN,1.0,NaN,...,1984-04-29,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22536,Justice Coal Of Alabama LLC,7,Mine #3,P3889,P3889r3,ALP3889,NaN,2.0,1.0,67.0,...,2008-03-13,NaT,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0
22537,Justice Coal Of Alabama LLC,7,Poore Mine,P3908,P3908r1,ALP3908,NaN,2.0,1.0,310.0,...,2009-06-10,NaT,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0
22538,D & E Mining LLC,7,Flat Rock Mine No 2,P3935,P3935r2,ALP3935,NaN,2.0,1.0,108.0,...,2010-05-17,NaT,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0
22539,Parker Processing LLC,1,Glade Preparation Plant,P3829,P3829r7,ALP3829,NaN,1.0,1.0,61.0,...,2002-11-20,NaT,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0


In [167]:
mines[mines['permit_id']=='3036']

,company,coalmine_op_status,mine_name,permit_id,state_tribe_mine_id,national_id,coal_bed_names,inspectable_unit_status,post_smcra,reported_area,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,Number_company_to_permit_id
974,TENNESSEE MINING INC,7,CUMBERLAND MINE #1,3036,NaN,TN3036,NaN,NaN,1.0,NaN,...,1998-09-30,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,2.0
3093,"Turner Coal Company, Inc.",5,NaN,3036,NaN,VA3036,NaN,NaN,NaN,11.5,...,NaT,2023-07-02,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0


In [177]:
mines[mines['permit_id']=='0480046']

,company,coalmine_op_status,mine_name,permit_id,state_tribe_mine_id,national_id,coal_bed_names,inspectable_unit_status,post_smcra,reported_area,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,Number_company_to_permit_id
3301,NaN,4,NaN,0480046,NaN,KY0480046,NaN,2.0,0.0,NaN,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0


In [187]:
check_nulls = pd.merge(mines, permit_id_missing_company, on=['permit_id'], how='inner').drop_duplicates().sort_values('permit_id', ascending=False)

In [188]:
file_name = 'check_nulls'
from google.colab import files
check_nulls.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [191]:
mines_clean = mines[mines['company'].isnull()]

In [192]:
pd.merge(df, mines_clean,  how='left', left_on=['orig_perm_id'], right_on = ['permit_id'])

,Company,start_date,Observations,orig_perm_id,mine_id,water,trees,grass,flooded_vegetation,crops,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,Number_company_to_permit_id
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,0.031113,0.623011,0.059563,0.032779,0.065345,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,0.031869,0.698788,0.039705,0.029274,0.039489,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,0.032205,0.539393,0.055222,0.030944,0.088415,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,MOUNT VICTORY COAL CO INC,2023-07-01,16,404774X,00000000000000003095,0.033029,0.735718,0.038324,0.029872,0.033263,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EVERGREEN MINING INC,2023-07-01,30,8260501,0000000000000000086a,0.030702,0.743186,0.033933,0.027343,0.029957,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32655,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,0.031039,0.587889,0.039089,0.025979,0.048085,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32656,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,0.030835,0.739765,0.032717,0.027254,0.030536,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32657,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,0.027965,0.710045,0.042984,0.025502,0.038912,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32658,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,0.036899,0.544176,0.040389,0.030385,0.046551,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dropping values with missing Company

In [179]:
mines = mines[[mines['company'].isnull()]]

In [ ]:
df = pd.merge(df, mines,  how='left', left_on=['orig_perm_id'], right_on = ['permit_id'])

In [194]:
file_name = 'all_mines_no_filter.csv'
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>